## Word2Vec

In [ ]:
import pandas
df = pandas.read_excel('https://raw.githubusercontent.com/ywchiu/sns_mining/master/data/yahoo_movie.xlsx', index_col=0)


In [ ]:

import jieba
corpus = []
for e in df['content'].tolist():
    corpus.append(list(jieba.cut(e)))

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.890 seconds.
DEBUG:jieba:Loading model cost 1.890 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [ ]:
from gensim.models import word2vec
size = 50  # 產生多少維度
min_count = 5 # 要算至少出現多少次數的字詞
workers = 1 # 使用多少個core 計算, -1 使用所有的core 進行計算
window = 10  # 上下文的區間
iter = 1000 # 神經網路訓練的迭代數
sample = 1e-5 # 取樣的數量

model = word2vec.Word2Vec(corpus,
                          workers = workers,
                          sample = sample,
                          vector_size = size,
                          min_count=min_count,
                          window = window,
                          epochs = iter)

In [ ]:
for ele in model.wv.key_to_index:
  print(ele)

In [ ]:
model.wv.most_similar('👍')

[('star', 0.6806949973106384),
 ('wars', 0.6485974192619324),
 ('the', 0.532431960105896),
 ('水準', 0.5186603665351868),
 ('a', 0.5169947147369385),
 ('短評', 0.4569917619228363),
 ('com', 0.4492340087890625),
 ('負評', 0.44025713205337524),
 ('每', 0.43012791872024536),
 ('商業片', 0.4260213077068329)]

In [ ]:
model.wv.get_vector('👍')

array([-0.8111984 ,  0.7824098 , -0.404319  , -0.30878487, -0.05896171,
       -0.7107195 ,  0.37326482, -0.03639407, -0.27496302, -0.9388743 ,
       -0.507287  ,  0.00281144,  0.06825916, -0.48068208, -0.00823188,
        0.97187495,  0.7206621 ,  1.7579001 , -0.3505318 , -0.7819903 ,
        0.04106327,  0.262528  ,  0.01717855, -0.3617978 , -0.49871975,
       -0.7581934 ,  0.7616043 , -0.8010093 , -0.3192664 , -0.42065609,
        1.0444846 , -0.45980304,  0.29721963, -0.07342824, -0.8796175 ,
        0.6161792 ,  1.2005955 ,  0.18912739, -0.1907141 ,  0.26831225,
       -0.09707648,  0.48874703,  0.30645916,  0.38452163,  1.5255069 ,
        0.8467447 ,  0.9346989 , -0.09029961,  0.9415535 ,  0.484221  ],
      dtype=float32)

In [ ]:
import jieba
corpus = []
tags = []
for idx,rec in df[df['status'].isin(['good', 'bad'])].iterrows():
  corpus.append(list(jieba.cut(rec.get('content'))))
  tags.append(rec.get('status'))

In [ ]:
import numpy as np
size = 50
cnt = 0
vec = np.zeros(size).reshape((1, size))
#print(vec.shape)
for word in corpus[0]:
  try:
    #print(model.wv.get_vector(word))
    vec += model.wv.get_vector(word)
    cnt += 1
  except:
    pass
vec / cnt

array([[ 0.3560753 ,  0.04139198,  0.31660726,  0.00827647, -0.15414863,
         0.49940599,  0.13376611,  0.3330899 , -0.2622231 ,  0.22831564,
        -0.43297082, -0.19021252, -0.1367482 , -0.1291772 , -0.29089187,
         0.17610502, -0.05112522, -0.54161985,  0.1948148 , -0.48770092,
        -0.61635255,  0.09163281,  0.05945749,  0.17196269,  0.57584158,
         0.34237506, -0.07874   , -0.37200043, -0.32271495,  0.03633143,
        -0.17765051, -0.12541925,  0.30536095,  0.01021841,  0.20397114,
         0.32990493,  0.08395853, -0.0181308 ,  0.28003842, -0.408849  ,
        -0.31137875, -0.01818421, -0.09850937, -0.17474771,  0.00719527,
         0.35448317, -0.35423775, -0.33748701, -0.06146792, -0.24803221]])

In [ ]:
import numpy as np
vecs = []
y = []
size = 50
for tag, s in zip(tags, corpus):
    vec = np.zeros(size).reshape((1, size))
    cnt = 0
    for w in s:
        if w in model.wv.key_to_index:
            vec += model.wv.get_vector(w)
            cnt += 1
    if cnt > 0:
      vecs.append(vec / cnt)
      y.append(tag)

In [ ]:
X = np.concatenate(vecs, axis = 0)


In [ ]:
X

array([[ 0.3560753 ,  0.04139198,  0.31660726, ..., -0.33748701,
        -0.06146792, -0.24803221],
       [ 0.19852904,  0.05250242,  0.43756859, ...,  0.12330433,
        -0.03953955, -0.0228153 ],
       [ 0.09542537, -0.35680254, -0.48725238, ...,  0.13814829,
        -0.06318698, -0.58010768],
       ...,
       [ 0.40745942,  0.57974166,  0.18223719, ...,  0.93339914,
         0.03467985, -0.01167483],
       [ 0.19487849,  0.18948247, -0.19573306, ...,  0.1892956 ,
         0.57415452, -0.33892492],
       [ 0.61334225, -0.0116849 ,  0.12783378, ..., -0.73565089,
        -0.28128726,  0.25704299]])

In [ ]:
# 建立訓練與測試資料集
from sklearn.model_selection import train_test_split
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
# 建立模型
from sklearn.svm import SVC
clf = SVC(kernel = 'linear')
clf.fit(train_X, train_y)

SVC(kernel='linear')

In [ ]:
pred_y = clf.predict(test_X)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(test_y,pred_y)

0.7564766839378239

In [ ]:
confusion_matrix(test_y,pred_y)

array([[54, 26],
       [21, 92]])